In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
data = pd.read_csv('data/properati-merge2017-2016-2015-SOLOSIX.csv')
test = pd.read_csv('data/properati_dataset_testing_noprice.csv')

In [3]:
data = data[data['state_name'].str.contains('G.B.A') | data['state_name'].str.contains('Capital Federal')]
test = test[test['state_name'].str.contains('G.B.A') | test['state_name'].str.contains('Capital Federal')]

In [4]:
def place_name_from_parent(x): ## Segun vi el 3 item del place es generalmente el lugar
    list = x.split('|')
    if(len(list) < 5):
        return np.nan
    else:
        return list[3]
    
def rescatar_barrios_no_oficiales(x):
    if x == 'Once':
        return ("Balvanera")
    elif x == 'Bajo Belgrano':
        return ("Belgrano")
    elif x == 'Alto Palermo':
        return ("Palermo")
    elif x == 'Tribunales':
        return ("San Nicolás")
    elif x == 'Las Cañitas':
        return("Palermo")
    elif x == 'Capital Federal':
        return(np.nan)
    else:
        return x
    
def recuperar_cant_amb(x):
    "Recupera el dato amb en x (puede ser la descripcion o el titulo)"
    try:
        pos = 0
        x = x.lower()
        if "ambientes" in x:
            pos = x.rfind("ambientes")
        elif "amb" in x:
            pos = x.rfind("amb")
        if pos != 0:
            try:
                return (np.float64(x[pos -2]))
            except ValueError:
                if ("dos amb" in x) or ("dos ambientes" in x):
                    return 2.0
                elif ("tres amb" in x) or ("tres ambientes" in x):
                    return 3.0
                elif ("cuatro amb" in x) or ("cuatro ambientes" in x):
                    return 4.0
                elif ("cinco amb" in x) or ("cinco ambientes" in x):
                    return 5.0
                elif ("seis amb" in x) or ("seis ambientes" in x):
                    return 6.0
                return np.nan
    except AttributeError:
        return None

In [5]:
data.loc[:,'place_name'] = data.place_with_parent_names.apply(place_name_from_parent) ## esto quita muchos problemas 
                                                                                    ## de place name como nombres de country
                                                                                    ## o palermo soho
test.loc[:,'place_name'] = test.place_with_parent_names.apply(place_name_from_parent)

In [6]:
data.loc[:,'place_name'] = data.place_name.apply(rescatar_barrios_no_oficiales)
test.loc[:,'place_name'] = test.place_name.apply(rescatar_barrios_no_oficiales)

In [7]:
data.dropna(axis=0,subset=['price_aprox_usd'],inplace=True) ## drop de los datos sin precio
data = data.loc[data['price_aprox_usd'] > 0,:]

In [8]:
data.loc[data.rooms.isnull(),'rooms'] = data.description.apply(recuperar_cant_amb)
data.loc[data.rooms.isnull(),'rooms'] = data.title.apply(recuperar_cant_amb)
data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.surface_covered_in_m2
test.loc[test.rooms.isnull(),'rooms'] = test.description.apply(recuperar_cant_amb)

In [9]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,1.872970e+05,168010.000000,168010.000000,2.358290e+05,2.358290e+05,2.358290e+05,2.201690e+05,2.120970e+05,1.600380e+05,2.089540e+05,32335.000000,163013.000000,3.103900e+04
mean,3.498834e+06,-34.554355,-58.523720,3.311677e+05,4.152422e+06,2.496151e+05,9.329406e+03,9.594269e+03,1.994335e+03,4.362411e+03,14.815680,2.916810,3.679427e+03
std,4.467474e+05,1.510308,1.227385,7.327490e+05,6.094209e+06,3.663019e+05,4.262381e+06,4.342732e+06,3.949270e+03,1.880112e+04,108.034819,1.384511,8.059029e+04
min,3.427208e+06,-51.626650,-122.419415,2.600000e+03,4.757708e+04,2.754180e+03,0.000000e+00,0.000000e+00,8.000000e-02,5.000000e-05,1.000000,0.000000,1.000000e+00
25%,3.429595e+06,-34.629148,-58.572162,1.035000e+05,1.602910e+06,9.700000e+04,4.600000e+01,4.500000e+01,1.107143e+03,1.533333e+03,2.000000,2.000000,9.100000e+02
50%,3.430787e+06,-34.593335,-58.478594,1.698500e+05,2.593742e+06,1.570000e+05,8.000000e+01,7.500000e+01,1.779897e+03,2.094622e+03,3.000000,3.000000,1.800000e+03
75%,3.435874e+06,-34.526359,-58.416391,3.200000e+05,4.654160e+06,2.800000e+05,1.960000e+02,1.550000e+02,2.428571e+03,2.878788e+03,6.000000,4.000000,3.600000e+03
max,6.693230e+06,52.619706,58.496000,1.234568e+08,7.984250e+08,5.000000e+07,2.000000e+09,2.000000e+09,1.271027e+06,4.000000e+06,7071.000000,40.000000,1.000150e+07


In [10]:
#Filtrar datos anomalos

In [11]:
# Nan para los datos faltantes es mejor que dropear todo el dato 
# Los nros estan masomenos pensados MASOMENOS JEJE
data.loc[(data['surface_total_in_m2']>5000) | (data['surface_total_in_m2'] < 30),'surface_total_in_m2'] = np.nan
data.loc[(data['surface_covered_in_m2']>5000) | (data['surface_covered_in_m2'] < 30),'surface_covered_in_m2'] = np.nan
data.loc[data['floor']>30,'floor'] = np.nan
data.loc[data['rooms']==0,'rooms'] = np.nan
data.loc[data['rooms']>10,'rooms'] = np.nan
data.loc[(data['lat']>-33) | (data['lat']<-36),['lat','lon','lat-lon']] = np.nan
data.loc[(data['lon']>-57) | (data['lon']<-60),['lat','lon','lat-lon']] = np.nan
data.loc[(data['expenses']>14000) | (data['expenses']<100),'expenses'] = np.nan

In [12]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,1.872970e+05,167338.000000,167338.000000,2.358290e+05,2.358290e+05,2.358290e+05,200710.000000,199315.000000,1.600380e+05,2.089540e+05,31637.000000,162788.000000,28503.000000
mean,3.498834e+06,-34.582784,-58.498742,3.311677e+05,4.152422e+06,2.496151e+05,200.652947,129.594225,1.994335e+03,4.362411e+03,4.457692,2.909551,2781.058941
std,4.467474e+05,0.110084,0.150541,7.327490e+05,6.094209e+06,3.663019e+05,317.000637,160.928681,3.949270e+03,1.880112e+04,3.855759,1.339991,2538.757301
min,3.427208e+06,-35.765171,-59.664010,2.600000e+03,4.757708e+04,2.754180e+03,30.000000,30.000000,8.000000e-02,5.000000e-05,1.000000,1.000000,100.000000
25%,3.429595e+06,-34.628759,-58.570991,1.035000e+05,1.602910e+06,9.700000e+04,52.000000,49.000000,1.107143e+03,1.533333e+03,2.000000,2.000000,1000.000000
50%,3.430787e+06,-34.593239,-58.478188,1.698500e+05,2.593742e+06,1.570000e+05,90.000000,80.000000,1.779897e+03,2.094622e+03,3.000000,3.000000,1900.000000
75%,3.435874e+06,-34.526761,-58.416216,3.200000e+05,4.654160e+06,2.800000e+05,210.000000,162.000000,2.428571e+03,2.878788e+03,6.000000,4.000000,3700.000000
max,6.693230e+06,-33.677756,-57.848496,1.234568e+08,7.984250e+08,5.000000e+07,5000.000000,5000.000000,1.271027e+06,4.000000e+06,30.000000,10.000000,14000.000000


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235829 entries, 0 to 264020
Data columns (total 23 columns):
id                            235829 non-null object
created_on                    235829 non-null object
property_type                 235829 non-null object
place_name                    230133 non-null object
place_with_parent_names       235829 non-null object
state_name                    235829 non-null object
geonames_id                   187297 non-null float64
lat-lon                       167338 non-null object
lat                           167338 non-null float64
lon                           167338 non-null float64
price                         235829 non-null float64
currency                      235829 non-null object
price_aprox_local_currency    235829 non-null float64
price_aprox_usd               235829 non-null float64
surface_total_in_m2           200710 non-null float64
surface_covered_in_m2         199315 non-null float64
price_usd_per_m2              160

In [14]:
# Dropeo las columnas que no se usan porque el test no las tiene o no son necesarias
data.drop(['price','price_usd_per_m2','price_per_m2','currency',\
           'id','created_on','geonames_id','lat-lon','price_aprox_local_currency',\
           'description','title','place_with_parent_names'], axis=1, inplace=True)
test.drop(['id','created_on','lat-lon','description','country_name'], axis=1, inplace=True)
#datos con mas de 5 nan los dropeo el 5 nro medio random
data.dropna(axis=0,thresh=5,inplace=True)

In [15]:
#Esto es importante, convierto los datos categoricos en numericos cada categoria una columna con 1 si true o 0 si false
##https://stackoverflow.com/questions/34265102/xgboost-categorical-variables-dummification-vs-encoding
##https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/
data = data.join(pd.get_dummies(data.property_type))
data.drop('property_type',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.place_name))
data.drop('place_name',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.state_name))
data.drop('state_name',axis=1, inplace=True)

In [16]:
#####XGBOOST
#https://github.com/dmlc/xgboost
#https://xgboost.readthedocs.io/en/latest/build.html
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
#https://www.slideshare.net/ShangxuanZhang/kaggle-winning-solution-xgboost-algorithm-let-us-learn-from-its-author


In [17]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
import scipy.stats as st

from sklearn.metrics import r2_score

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
#divide el set de datos en parte de entrenamiento y parte de test
# no se puede usar el test bajado de kaggle porque no tiene el precio
X = data.drop(["price_aprox_usd"], axis=1)
Y = data["price_aprox_usd"]
X, X_Val, Y, Y_Val = train_test_split(X, Y)

In [20]:
## Random search de hiperparametros y entrenamiento del algoritmo, 
## cuando termina tira dos scores (no se cual es mejor) y los mejores hiperparametros que dieron esos scores
## ES LA PARTE QUE TARDA
#http://danielhnyk.cz/how-to-use-xgboost-in-python/
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": st.randint(3, 40),#nros random desde 30 hasta 40
    "max_depth": st.randint(3, 40),#nros random desde 30 hasta 40
    "learning_rate": st.uniform(0.05, 0.4), # nros random de una distribucion uniforme entre 0.1 y 0.1 +0.05
    "colsample_bytree": one_to_left, #cosa rara, no la entendi todavia algo de la dist beta
    "subsample": one_to_left, 
    "gamma": st.uniform(0, 10),
    'reg_alpha': from_zero_positive,#cosa rara, no la entendi todavia algo de la dist exponencial
    "min_child_weight": from_zero_positive,
}

xgb = XGBRegressor(nthread=-1) 

grid = RandomizedSearchCV(xgb, params, n_jobs=1,n_iter=10) # n_iters es la cant de veces que busca, 10 es lo default
grid.fit(X, Y)
print(r2_score(Y_Val, grid.best_estimator_.predict(X_Val)))
print(grid.best_score_)
print(grid.best_estimator_)
grid.best_params_

0.672171945703
0.538965230845
XGBRegressor(base_score=0.5, colsample_bylevel=1,
       colsample_bytree=0.94060921190816416, gamma=4.3960685341984682,
       learning_rate=0.17383769900746918, max_delta_step=0, max_depth=38,
       min_child_weight=17.593339441716235, missing=None, n_estimators=34,
       nthread=-1, objective='reg:linear', reg_alpha=71.168216535846256,
       reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=0.96513611400596888)


{'colsample_bytree': 0.94060921190816416,
 'gamma': 4.3960685341984682,
 'learning_rate': 0.17383769900746918,
 'max_depth': 38,
 'min_child_weight': 17.593339441716235,
 'n_estimators': 34,
 'reg_alpha': 71.168216535846256,
 'subsample': 0.96513611400596888}

In [ ]:
# Parte para probar los parametros sin random search
# no tarda mucho porque tira el score de la combinacion param que le pones nada mas
xgb = XGBRegressor(colsample_bytree=0.9,gamma=4,learning_rate=0.15,max_depth=39,min_child_weight=18,n_estimators=35,\
                  reg_alpha=8,subsample=0.99) 

xgb.fit(X, Y,eval_metric="rmse")

# Print the r2 score
print(r2_score(Y_Val, xgb.predict(X_Val)))